In [1]:
%matplotlib notebook
import ipywidgets as widgets
from IPython.display import display
from traitlets import Unicode, validate, List
import pandas as pd
import json

class HelloWidget(widgets.DOMWidget):
    _view_name = Unicode('HelloView').tag(sync=True)
    _view_module = Unicode('hello').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)
    value = List([]).tag(sync=True)
    count = List([]).tag(sync=True)

In [2]:
%%javascript
require.config({
    paths: {
        d3: 'https://d3js.org/d3.v4.min'
    }
});

<IPython.core.display.Javascript object>

In [3]:
%%javascript
require.undef('hello');
define('hello', ["@jupyter-widgets/base", "d3"], function(widgets, d3) {

    var HelloView = widgets.DOMWidgetView.extend({

        render: function() {
            this.value_changed();
            this.model.on('change:value', this.value_changed, this);
        },

        value_changed: function() {
            console.log("check");
            var player = this.model.get('value');
            var that = this;
            var test = this.model.get('count');
//             alert(test);
var yearValues = [];
var attribValues = [];
var playerYearDataList = [];
player.sort(function(x, y){
    return d3.ascending(x[2], y[2]);
})
console.log(player);
player.forEach(function(d){
    yearValues.push(d["year"]);
    attribValues.push(d["overall_rating"]);
})
console.log(yearValues);
console.log(attribValues);
var margin = {top: 10, right: 30, bottom: 30, left: 50};
var svgHeight = 400;
var svgWidth = 1000;
  //create canvas
  $("#chart1").remove();
  this.$el.append("<div id='chart1'></div>");
  $("#chart1").width("960px");
  $("#chart1").height("600px");        
  var margin = {top: 20, right: 20, bottom: 30, left: 40};
  var width = 880 - margin.left - margin.right;
  var height = 500 - margin.top - margin.bottom;
  var svg = d3.select("#chart1").append("svg")
    .style("position", "relative")
    .style("max-width", "960px")
    .attr("width", width + "px")
    .attr("height", (height + 50) + "px");
  svg.append('g')
     .attr("id", "xAxis");
  svg.append('g')
     .attr("id", "yAxis");
    console.log(d3);
  let yScale = d3.scaleLinear()
            .domain([d3.min(attribValues, d => d), d3.max(attribValues, d => d)])
            .range([svgHeight - margin.top - margin.bottom, 0]);

        let yAxis = d3.axisLeft();
        // assign the scale to the axis
        yAxis.scale(yScale);


        var yAxisG = d3.select("#yAxis")
            .attr("transform", "translate("+margin.left+"," + margin.top +")");
        // self.svg.append("g")
        // .attr("class" , "yAxis");

        console.log(yAxisG);

        yAxisG.transition(3000).call(yAxis);


        let xScale = d3.scaleLinear()
            .domain([d3.min(yearValues), d3.max(yearValues)])
            .range([0, 600]);

        let xAxis = d3.axisBottom();
        // assign the scale to the axis
        xAxis.scale(xScale);


        var xAxisG = d3.select("#xAxis")
            .attr("transform", "translate("+(margin.left+ 10)+"," + (svgHeight - margin.bottom) +")");
        // self.svg.append("g")
        // .attr("class" , "yAxis");

        console.log(xAxisG);
//         let color = d3.scaleLinear()
//             .domain([0, playerYearDataList.length])
//             // .range(["#016450", "#ece2f0"]);
//             .range(["#2019F6", "#F61936"]);

        xAxisG.transition(3000).call(xAxis);
        svg.selectAll(".playerPath").remove();
        svg.selectAll(".playerNode").remove();
            console.log(player.name);
            console.log(player.playerYearData);

            var lineCoords = []
            for(var k=0; k<yearValues.length; k++){
                lineCoords.push([xScale(yearValues[k]), yScale(attribValues[k])]);
            }

            console.log(lineCoords);

            var lineGenerator = d3.line();
            var pathString = lineGenerator(lineCoords);

            console.log(pathString);

            svg.append('path')
                .attr('d', pathString)
                .attr("transform", "translate("+(margin.left+ 10)+"," + (margin.top) +")")
                .attr("style", "fill : none;")
                .attr("class", "playerPath")
                .style("stroke", "steelblue")
                .style("stroke-width", 3)
                .style('opacity', 0.5);
            
            lineCoords.forEach(function(point){
                svg.append('circle').attr('cx', point[0])
                    .attr("cy", point[1])
                    .attr("r", 5)
                    .attr("transform", "translate("+(margin.left+ 10)+"," + (margin.top) +")")
                    .attr("class", "playerNode");
            });
            d3.selectAll(".brush").remove();
        var brush = d3.brushX().extent([[margin.left,svgHeight-margin.bottom-20],[svgWidth,svgHeight-10]]).on("end", brushed);

        svg.append("g").attr("class", "brush").call(brush);


        function brushed() {

            var sel = d3.event.selection;

            if(sel === null){
                return;
            }

            var yearValuesBrushed = yearValues.filter((d) => xScale(d)+margin.left+ 10 >= sel["0"] &&  xScale(d)+margin.left+ 10  <= sel["1"]);

            

            console.log(yearValuesBrushed);
            that.model.set('count', yearValuesBrushed);
            that.model.save_changes();
            that.touch();
//             alert(that.model.get('test'));

            // var dataSel = self.posd.filter((d) => d.position >= sel["0"] && d.position <= sel["1"]);
            // window.selectedStatesIn = dataSel.map( d => d.elem);
            // self.shiftChart.update(window.selectedStatesIn, window.selectedYearsIn);
//             self.updateBars(playerYearDataList, uniqueYrs , attrib, color);


        }
        },
    });

    return {
        HelloView : HelloView
    };
});

<IPython.core.display.Javascript object>

In [4]:
player_name = "Alexis Sanchez"
playersData = pd.read_csv(open("player_data.csv"))
def filter_data(name, attr):
    filterByName = playersData[playersData["player_name"]==name]
    returnData = filterByName[["player_name", attr, "year"]]
    return returnData
players_data = filter_data(player_name, "overall_rating")

In [5]:
w = HelloWidget()
names = ['Alexis Sanchez', 'Andres Iniesta', 'Angel Di Maria', 'Arjen Robben']
def update(name):
    w.value = filter_data(name['new'], "overall_rating").to_dict(orient='records')
def createDropdown(names):
    mark_choose = widgets.Dropdown(options=names, description='Player')
    mark_choose.observe(update, names='value')
    display(mark_choose)
createDropdown(names)
display(w)
update({'new': player_name})

Dropdown(description='Player', options=('Alexis Sanchez', 'Andres Iniesta', 'Angel Di Maria', 'Arjen Robben'), value='Alexis Sanchez')

HelloWidget()

value
[24, 27]


In [8]:
def updateTest(change):
    print(change['new'])  
w.observe(updateTest, names=['count'])

In [12]:
def func(change):
    print('value')   # as of traitlets 4.3, one should be able to
                           # write print(change.new) instead
    w.count = [24, 27]    

w.observe(func, names=['value'])
w.count

[2009, 2010]

In [ ]:
def any_function_returning_value(word1,word2):
        new_word = 'Combining words is easy: %s %s'%(word1,word2)
        print(new_word)
        return new_word

class learn_classes_and_widgets():
    def __init__(self, param1 = 'a word', param2 = 'another word'):
        self.p1_text = widgets.Text(description = 'Word #1',value = param1)
        self.p2_text = widgets.Text(description = 'Word #2',value = param2)
        self.p1_text.on_submit(self.handle_submit)
        self.p2_text.on_submit(self.handle_submit)
        display(self.p1_text, self.p2_text)

    def handle_submit(self, text):
        print("Submitting")
        self.w = any_function_returning_value(self.p1_text.value,self.p2_text.value)
        return self.w

f = learn_classes_and_widgets(param1 = 'try this word')
    #f.w should contain the combined words when this is all working